# Data Preparation - Dataset de Sepsis
## CRISP-DM Fase 3: Preparação dos Dados

**Objetivo da Fase:**
* Transformar dados brutos em formato adequado para modelagem
* Implementar estratégias de limpeza e tratamento baseadas nos insights da EDA
* Criar features derivadas com relevância clínica
* Preparar datasets finais para algoritmos de machine learning

**Baseado nos Insights da EDA:**
* 37/41 variáveis apresentam missing values (68.37% do dataset)
* 27 variáveis com >80% missing (candidatas à remoção)
* Dataset altamente desbalanceado: 98.2% não-sepsis vs 1.8% sepsis
* Estrutura temporal importante: risco aumenta após 100h na UTI
* Variáveis categóricas bem definidas: Gender, Unit1, Unit2

**Tarefas CRISP-DM a serem executadas:**
1. **Seleção dos Dados**: Escolher variáveis mais relevantes
2. **Limpeza dos Dados**: Tratar inconsistências e valores ausentes  
3. **Construção dos Dados**: Criar features derivadas e engenharia
4. **Integração dos Dados**: Combinar fontes (não aplicável aqui)
5. **Formatação dos Dados**: Preparar formato final para modelagem

## Configuração do Ambiente Google Colab

Para funcionar no Google Colab, é necessário criar um atalho do diretório MDA no seu próprio Drive e então rodar os dois comandos abaixo e conceder permissão ao seu drive quando rodar a célula logo abaixo.

[Link](https://towardsdatascience.com/simplify-file-sharing-44bde79a8a18/) detalhando como funciona

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# modificar para o diretorio que contém os dados de teste e treino
%cd /content/drive/MyDrive/MDA/Train\ and\ test\ data\ -\ Proj\ DM/

!ls

## 1. Importação das Bibliotecas

Importação de todas as bibliotecas necessárias para preparação dos dados, incluindo bibliotecas específicas para pré-processamento, feature engineering e balanceamento de classes.

In [ ]:
# Bibliotecas essenciais para manipulação de dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Bibliotecas para pré-processamento
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif

# Bibliotecas para balanceamento de classes
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek

# Configurações gerais
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("Bibliotecas importadas com sucesso")

## 2. Carregamento dos Dados e Insights da EDA

Carregamento dos datasets de treino e teste, seguido da documentação dos principais insights obtidos na análise exploratória que guiarão as decisões de preparação.

### 2.1 Carregamento dos Datasets

In [ ]:
# Carregar datasets de treino e teste
print("Carregando datasets...")

# Dataset de treino
train_df = pd.read_csv('dataset_sepsis_train.csv')
print(f"Dataset de treino: {train_df.shape}")

# Dataset de teste  
test_df = pd.read_csv('dataset_sepsis_test.csv')
print(f"Dataset de teste: {test_df.shape}")

# Separar features e target
X_train = train_df.drop('SepsisLabel', axis=1)
y_train = train_df['SepsisLabel']

X_test = test_df.drop('SepsisLabel', axis=1) 
y_test = test_df['SepsisLabel']

print(f"\nFormas finais:")
print(f"X_train: {X_train.shape} | y_train: {y_train.shape}")
print(f"X_test: {X_test.shape} | y_test: {y_test.shape}")

# Verificar distribuição das classes
print(f"\nDistribuição das classes no treino:")
print(y_train.value_counts(normalize=True))

### 2.2 Documentação dos Insights da EDA

Resumo dos principais achados da análise exploratória que orientarão as decisões de preparação dos dados.

In [ ]:
# Documentar insights chave da EDA para orientar a preparação
eda_insights = {
    'missing_patterns': {
        'very_high_missing': '27 variáveis com >80% missing (candidatas à remoção)',
        'medium_missing': '4 variáveis com 20-80% missing (imputação específica)',
        'low_missing': '6 variáveis com <20% missing (imputação simples)',
        'complete_vars': '4 variáveis sem missing values'
    },
    
    'temporal_patterns': {
        'risk_progression': 'Risco de sepsis aumenta após 100h na UTI',
        'data_concentration': '75% das observações nas primeiras 75h',
        'temporal_dependency': 'Múltiplas observações por paciente ao longo do tempo'
    },
    
    'class_imbalance': {
        'ratio': '98.2% não-sepsis vs 1.8% sepsis',
        'imbalance_factor': '54.7:1',
        'strategy_needed': 'Balanceamento obrigatório para modelagem'
    },
    
    'feature_quality': {
        'categorical_vars': ['Gender', 'Unit1', 'Unit2'],
        'high_separability': ['Lactate', 'Creatinine', 'BUN'],
        'temporal_features': ['Hour', 'HospAdmTime', 'ICULOS']
    }
}

print("INSIGHTS DA EDA DOCUMENTADOS:")
for category, insights in eda_insights.items():
    print(f"\n{category.upper().replace('_', ' ')}:")
    for key, value in insights.items():
        print(f"  • {key}: {value}")

## 3. TAREFA 1: Seleção dos Dados

**Objetivo:** Escolher as variáveis mais relevantes para o modelo de mineração, removendo features com baixo potencial preditivo ou problemas graves de qualidade.

**Critérios de seleção:**
* Relevância clínica para detecção de sepsis
* Percentual de missing values aceitável
* Separabilidade entre classes (baseada na EDA)
* Variabilidade e distribuição adequadas

### 3.1 Remoção de Variáveis com Excesso de Missing Values

Identificação e remoção de variáveis com >95% de valores ausentes, que oferecem pouco valor preditivo devido à escassez de dados.

In [ ]:
# Placeholder para implementação da remoção de variáveis com alto missing
# Será implementado baseado nos achados da EDA
pass

### 3.2 Seleção por Relevância Clínica

Manutenção de variáveis clinicamente importantes para sepsis, mesmo com missing values moderados, priorizando biomarcadores e sinais vitais críticos.

In [ ]:
# Placeholder para seleção baseada em critérios clínicos
# Definir lista de features prioritárias baseada em literatura médica
pass

### 3.3 Análise de Separabilidade Estatística

Avaliação da capacidade discriminativa das variáveis restantes usando testes estatísticos e métricas de separação entre classes.

In [ ]:
# Placeholder para análise estatística de separabilidade
# Implementar testes de hipótese e métricas de discriminação
pass

## 4. TAREFA 2: Limpeza dos Dados

**Objetivo:** Corrigir ou remover dados inconsistentes, duplicados ou ausentes através de estratégias específicas para cada tipo de variável.

**Estratégias por tipo de missing:**
* Missing < 20%: Imputação simples (mediana/moda)
* Missing 20-80%: Imputação específica por domínio clínico
* Missing > 80%: Avaliação caso a caso para remoção

### 4.1 Detecção e Remoção de Duplicatas

Identificação de registros duplicados exatos e tratamento adequado considerando a natureza temporal dos dados.

In [ ]:
# Placeholder para detecção de duplicatas
# Considerar aspectos temporais na identificação
pass

### 4.2 Tratamento de Outliers

Identificação de valores extremos usando critérios estatísticos e clínicos, com decisões de remoção ou transformação baseadas na relevância médica.

In [ ]:
# Placeholder para tratamento de outliers
# Implementar métodos estatísticos (IQR, Z-score) e critérios clínicos
pass

### 4.3 Estratégias de Imputação

Implementação de diferentes técnicas de imputação baseadas no tipo de variável e percentual de missing values.

#### 4.3.1 Imputação para Variáveis com Baixo Missing (<20%)

Aplicação de imputação simples usando medidas centrais apropriadas para cada tipo de variável.

In [ ]:
# Placeholder para imputação simples
# Mediana para numéricas, moda para categóricas  
pass

#### 4.3.2 Imputação Avançada para Missing Moderado (20-80%)

Uso de técnicas mais sofisticadas como KNN Imputer ou imputação baseada em modelos para variáveis clinicamente importantes.

In [ ]:
# Placeholder para imputação avançada
# KNN Imputer, imputação por regressão
pass

### 4.4 Validação da Qualidade Pós-Limpeza

Verificação da qualidade dos dados após as operações de limpeza, incluindo distribuições e consistência lógica.

In [ ]:
# Placeholder para validação pós-limpeza
# Verificar distribuições, ranges aceitáveis, consistência
pass

## 5. TAREFA 3: Construção dos Dados (Feature Engineering)

**Objetivo:** Criar novas variáveis ou atributos derivados dos dados existentes que possam melhorar o poder preditivo do modelo.

**Estratégias de construção:**
* Ratios clínicos baseados em literatura médica
* Features temporais derivadas de Hour/ICULOS
* Interações entre variáveis relacionadas
* Transformações para normalizar distribuições

### 5.1 Criação de Ratios Clínicos

Desenvolvimento de índices e ratios clinicamente estabelecidos para detecção de sepsis (ex: razão neutrófilos/linfócitos, índices de choque).

In [ ]:
# Placeholder para criação de ratios clínicos
# Implementar ratios estabelecidos na literatura médica
pass

### 5.2 Features Temporais

Criação de variáveis derivadas das informações temporais para capturar padrões de risco ao longo do tempo.

In [ ]:
# Placeholder para features temporais
# Categorização por janelas de risco, tendências temporais
pass

### 5.3 Interações entre Variáveis

Criação de features que capturam interações sinérgicas entre variáveis clínicas relacionadas.

In [ ]:
# Placeholder para features de interação
# Produtos, somas ponderadas, interações não-lineares
pass

### 5.4 Transformações de Distribuição

Aplicação de transformações matemáticas para normalizar distribuições assimétricas identificadas na EDA.

In [ ]:
# Placeholder para transformações
# Log, Box-Cox, transformações de potência
pass

## 6. TAREFA 4: Integração dos Dados

**Objetivo:** Combinar dados de diferentes fontes em um único conjunto de dados consistente.

**Observação:** Esta tarefa não se aplica ao dataset atual, pois trabalhamos com uma única fonte (PhysioNet 2019 Challenge). Esta seção documenta a estrutura para futuras expansões do projeto.

### 6.1 Documentação da Fonte Única

Registro da origem e características do dataset único utilizado no projeto.

In [ ]:
# Documentar características da fonte única
dataset_info = {
    'source': 'PhysioNet 2019 Challenge',
    'description': 'Early Detection of Sepsis from Clinical Data',
    'patients': 'Multiple ICU patients with temporal observations',
    'timeframe': 'Variable length ICU stays',
    'completeness': 'Single comprehensive source'
}

print("INFORMAÇÕES DA FONTE DE DADOS:")
for key, value in dataset_info.items():
    print(f"  • {key}: {value}")

### 6.2 Preparação para Futuras Integrações

Estrutura preparatória para possível integração com outras fontes de dados em versões futuras do projeto.

In [ ]:
# Placeholder para estrutura de integração futura
# Definir schemas, chaves de junção, protocolos de merge
pass

## 7. TAREFA 5: Formatação dos Dados

**Objetivo:** Preparar os dados no formato necessário para os algoritmos de modelagem, incluindo normalização, encoding e divisão final dos conjuntos.

**Atividades principais:**
* Normalização/padronização de variáveis numéricas
* Encoding de variáveis categóricas  
* Balanceamento de classes
* Criação dos datasets finais para modelagem

### 7.1 Normalização e Padronização

Aplicação de transformações de escala para garantir que todas as variáveis numéricas tenham contribuições equilibradas nos algoritmos.

In [ ]:
# Placeholder para normalização
# StandardScaler, MinMaxScaler baseado na distribuição das variáveis
pass

### 7.2 Encoding de Variáveis Categóricas

Conversão de variáveis categóricas para formato numérico apropriado para algoritmos de machine learning.

In [ ]:
# Placeholder para encoding categórico
# One-hot encoding, label encoding baseado na cardinalidade
pass

### 7.3 Balanceamento de Classes

Implementação de técnicas para lidar com o severo desbalanceamento entre classes (98.2% vs 1.8%).

#### 7.3.1 Análise de Estratégias de Balanceamento

Comparação de diferentes abordagens: oversampling, undersampling e métodos combinados.

In [ ]:
# Placeholder para comparação de estratégias
# SMOTE, Random Over/Under Sampling, SMOTETomek
pass

#### 7.3.2 Implementação da Estratégia Escolhida

Aplicação da técnica de balanceamento selecionada com base na análise comparativa.

In [ ]:
# Placeholder para implementação do balanceamento
# Aplicar técnica escolhida e validar resultados
pass

### 7.4 Criação dos Datasets Finais

Montagem dos conjuntos de dados finais prontos para a fase de modelagem, incluindo validação da integridade.

In [ ]:
# Placeholder para datasets finais
# Criar X_train_final, X_test_final, y_train_final, y_test_final
pass

### 7.5 Validação Final e Export

Verificação final da qualidade e consistência dos dados preparados, seguida do salvamento dos datasets processados.

In [ ]:
# Placeholder para validação final
# Verificar shapes, tipos, ranges, consistência lógica
pass

## 8. Resumo da Preparação

**Síntese das transformações aplicadas:** Documentação completa de todas as modificações realizadas nos dados durante o processo de preparação.

**Datasets resultantes:** Características finais dos conjuntos de dados prontos para modelagem.

**Próximos passos:** Direcionamento para a fase de modelagem do CRISP-DM.

In [ ]:
# Placeholder para resumo final
# Documentar todas as transformações e características finais
pass

---

**Próxima Fase:** Modeling (3-modeling.ipynb)

**Entregáveis desta fase:**
- Datasets limpos e preparados
- Features engineered com relevância clínica  
- Classes balanceadas adequadamente
- Documentação completa das transformações
- Validação da qualidade dos dados processados